


<h1>QGAN using PennyLane</h1>



In [ ]:
pip install pennylane

     |████████████████████████████████| 880 kB 6.8 MB/s 
     |████████████████████████████████| 1.6 MB 25.1 MB/s 
     |████████████████████████████████| 8.5 MB 20.3 MB/s 
     |████████████████████████████████| 108 kB 55.0 MB/s 


In [ ]:
import pennylane as qml
import numpy as np
import tensorflow as tf

In [ ]:
pip install pennylane-qiskit pennylane-cirq

     |████████████████████████████████| 3.4 MB 14.5 MB/s 
     |████████████████████████████████| 238 kB 70.9 MB/s 
     |████████████████████████████████| 6.5 MB 56.8 MB/s 
     |████████████████████████████████| 255 kB 65.9 MB/s 
     |████████████████████████████████| 18.0 MB 77 kB/s 
     |████████████████████████████████| 200 kB 48.8 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 943 kB 49.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 49 kB 3.4 MB/s 
     |████████████████████████████████| 37.5 MB 40 kB/s 
     |████████████████████████████████| 3.6 MB 52.3 MB/s 
     |████████████████████████████████| 113 kB 78.1 MB/s 
     |████████████████████████████████| 1.8 MB 25.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.35.0-py3-none-any.whl size=11839 sha256=1122323283f80d0518b9b73081518339f0cb40a58e58fc35d7e9

In [ ]:
dev = qml.device('cirq.simulator', wires=3)

In [ ]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

In [ ]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

In [ ]:

def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [ ]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))


@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

In [ ]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

# Training the QGAN

In [ ]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

In [ ]:
print(np.pi )

3.141592653589793


In [ ]:
print(gen_weights,disc_weights)

<tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 3.15923318e+00,  4.00157208e-03,  9.78737984e-03,  2.24089320e-02,
        1.86755799e-02, -9.77277880e-03,  9.50088418e-03, -1.51357208e-03,
       -1.03218852e-03])> <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 0.4105985 ,  0.14404357,  1.45427351,  0.76103773,  0.12167502,
        0.44386323,  0.33367433,  1.49407907, -0.20515826])>


In [ ]:
opt = tf.keras.optimizers.SGD(0.4)

In [ ]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.05727693438529968
Step 5: cost = -0.2634810581803322
Step 10: cost = -0.42739150300621986
Step 15: cost = -0.4726157411932945
Step 20: cost = -0.48406897485256195
Step 25: cost = -0.48946385085582733
Step 30: cost = -0.4928187355399132
Step 35: cost = -0.49494922906160355
Step 40: cost = -0.4962702915072441
Step 45: cost = -0.4970718249678612


In [ ]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

Prob(real classified as real):  0.9985871016979218


In [ ]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.5011128559708595


In [ ]:
cost = lambda: gen_cost(gen_weights)

for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.5833386555314064
Step 5: cost = -0.8915733397006989
Step 10: cost = -0.9784241914749146
Step 15: cost = -0.9946482479572296
Step 20: cost = -0.9984995126724243
Step 25: cost = -0.9995638430118561
Step 30: cost = -0.9998717308044434
Step 35: cost = -0.9999619722366333
Step 40: cost = -0.9999887645244598
Step 45: cost = -0.9999964833259583


In [ ]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.9999986886978149


In [ ]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.21694186  0.45048442 -0.86602521]
Generator Bloch vector: [-0.28404659  0.41893226 -0.86244404]
